# SpatialDE: spatially variable genes report

In [ ]:
import os
print("Files in use:")
print(f"FILE='{os.path.abspath(FILE)}'")

In [ ]:
# Import packages
import scanpy as sc
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import warnings
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import rc_context

#### Plotting settings

In [ ]:
# plot settings
sc.set_figure_params(dpi=150, fontsize=10, dpi_save=600)
hist_dims = (10,3)

### Read data

In [ ]:
adata = sc.read_h5ad(filename=FILE)

---

## Spatially variable genes

#### Volcano plots: adjusted p-values vs. fraction spatial variance
Volcano plots can be investigated to properly set thresholds for max. adjusted p-value and min. fraction of spatial variance (FSV) to select spatially variable genes

In [ ]:
small_float_cutoff = 10**-12

adjustment_method = adata.uns['spatialDE_parameters']['qval_adjust_method']
thr_qval = adata.uns['spatialDE_parameters']['thr_qval']
min_fsv = adata.uns['spatialDE_parameters']['min_fsv']

plt.yscale('log')
plt.scatter(adata.uns['spatialDE']['FSV'], [small_float_cutoff if qval < small_float_cutoff else qval for qval in adata.uns['spatialDE'][adjustment_method]], c='black')

plt.axhline(thr_qval, c='red', lw=1, ls='--')
plt.axvline(min_fsv, c='red', lw=1, ls='--')

plt.gca().invert_yaxis();

plt.xlabel('Fraction spatial variance')
plt.ylabel('Adj. P-value:' + adjustment_method);

In [ ]:
print("Number of significant genes: {}".format(np.sum(adata.uns['spatialDE']['is_signif'])))

#### Top candidate genes
To select the most interesting genes, spatialDE results can be ranked by log-likelihood ratio (LLR) 

In [ ]:
adata.uns['spatialDE'].sort_values('LLR', ascending=False).head(n=20)

#### Expression of top 8 genes
This data should have been normalized with NaiveDE. Hence, raw counts as used in SCope session may show some deviations. 

In [ ]:
top8_genes = adata.uns['spatialDE'].sort_values('LLR', ascending=False)[0:8].g

if 'spatial' in adata.uns.keys():
    sc.pl.spatial(adata, img_key="hires", color=top8_genes, alpha=1, color_map='OrRd')
else:
    fig_width= 10
    sp_width = np.max(adata.obsm['X_spatial'][:,0]) - np.min(adata.obsm['X_spatial'][:,0])
    sp_height = np.max(adata.obsm['X_spatial'][:,1]) - np.min(adata.obsm['X_spatial'][:,1])
    ratio_hw = sp_height / sp_width
    fig_height = int(ratio_hw * fig_width)
    with rc_context({'figure.figsize': (fig_width, fig_height)}):
        sc.pl.embedding(adata, basis="X_spatial", color=top8_genes, alpha=1, color_map='OrRd')